# Import Packages 

In [1]:
#import data
import pandas as pd
import great_expectations as gx
import os

# Importing Data, Quick Look Through

In [2]:
#Devices

In [3]:
devices=pd.read_csv("/home/neobanking2025/projects/neo_bank/raw_data/devices.csv")
devices.head()

,string_field_0,string_field_1
0,Apple,user_6809
1,Apple,user_3636
2,Apple,user_510
3,Apple,user_3468
4,Apple,user_7036


In [4]:
len(devices)

19431

In [5]:
devices=devices.drop_duplicates()

In [6]:
len(devices)

19431

In [7]:
devices.describe()

,string_field_0,string_field_1
count,19431,19431
unique,4,19431
top,Android,user_6809
freq,9714,1


In [8]:
devices.rename(columns={
    "string_field_0": "device_type",
    "string_field_1": "user_id"
},inplace=True)

In [9]:
devices.head()

,device_type,user_id
0,Apple,user_6809
1,Apple,user_3636
2,Apple,user_510
3,Apple,user_3468
4,Apple,user_7036


In [10]:
#Transactions - very big file, will try loading from parquet cache

In [11]:
csv_path = "/home/neobanking2025/projects/neo_bank/raw_data/transactions.csv"
parquet_path = "/home/neobanking2025/projects/neo_bank/raw_data/transactions.parquet"

if os.path.exists(parquet_path):
    # Load from Parquet cache — fast!
    trans = pd.read_parquet(parquet_path)
else:
    # First time: read CSV (slow)
    trans = pd.read_csv(csv_path)
    # Save cache to Parquet for next time
    trans.to_parquet(parquet_path, index=False)

In [12]:
trans.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_1884,REFUND,AED,1.76,COMPLETED,None,NaN,None,None,INBOUND,user_8098,2018-09-25 16:03:40.978243 UTC
1,transaction_1728,REFUND,AED,639.31,COMPLETED,None,NaN,None,None,INBOUND,user_182,2018-03-31 13:45:25.262231 UTC
2,transaction_1755782,REFUND,AED,3.49,COMPLETED,None,NaN,None,None,INBOUND,user_14271,2018-11-17 16:47:25.748975 UTC
3,transaction_1973,TOPUP,AED,6079.39,COMPLETED,None,NaN,None,None,INBOUND,user_4773,2018-08-13 21:51:28.670363 UTC
4,transaction_1716,TOPUP,AED,206.65,COMPLETED,None,NaN,None,None,INBOUND,user_1363,2018-10-30 14:34:57.479895 UTC


In [13]:
len(trans)

2740075

In [14]:
trans=trans.drop_duplicates()

In [15]:
len(trans)

2740075

In [16]:
trans.describe()

,amount_usd,ea_merchant_mcc
count,2.740075e+06,1.581417e+06
mean,1.703225e+05,5.699736e+03
std,9.634014e+07,9.493660e+02
min,0.000000e+00,7.420000e+02
25%,2.030000e+00,5.411000e+03
50%,8.510000e+00,5.812000e+03
75%,2.822000e+01,5.921000e+03
max,8.510645e+10,9.406000e+03


In [17]:
#users

In [18]:
users=pd.read_csv("/home/neobanking2025/projects/neo_bank/raw_data/users.csv")
users.head()

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals
0,user_2811,1988,IE,C,2018-03-04 21:46:39.616834 UTC,0,STANDARD,NaN,NaN,0,0,0
1,user_4750,1987,FR,Ry,2018-04-07 06:49:18.345736 UTC,0,STANDARD,NaN,NaN,0,0,0
2,user_17686,1984,GB,Ayr,2018-12-04 10:28:02.653147 UTC,0,PREMIUM,NaN,NaN,6,0,0
3,user_18779,1965,GB,Ayr,2018-12-31 07:42:19.353613 UTC,0,STANDARD,NaN,NaN,0,0,0
4,user_7823,1999,PL,Buk,2018-06-07 03:22:40.590209 UTC,0,STANDARD,NaN,NaN,0,0,0


In [19]:
len(users)

19430

In [20]:
users=users.drop_duplicates()

In [21]:
len(users)

19430

In [22]:
#Notifications

In [23]:
notifs=pd.read_csv("/home/neobanking2025/projects/neo_bank/raw_data/notifications.csv")
notifs.head()

,reason,channel,status,user_id,created_date
0,METAL_RESERVE_PLAN,SMS,SENT,user_4703,2018-10-14 01:48:13.319987 UTC
1,METAL_RESERVE_PLAN,SMS,SENT,user_2397,2018-10-17 08:07:05.709072 UTC
2,METAL_RESERVE_PLAN,SMS,SENT,user_2411,2018-10-14 03:46:34.244392 UTC
3,METAL_RESERVE_PLAN,SMS,SENT,user_1119,2018-10-14 14:16:35.801185 UTC
4,METAL_RESERVE_PLAN,SMS,SENT,user_3499,2018-10-14 18:41:09.268675 UTC


In [24]:
len(notifs)

121813

In [25]:
notifs=notifs.drop_duplicates()

In [26]:
len(notifs)

121813

# Creating Expectations

In [27]:
context = gx.get_context()

## Devices

In [28]:
devices.head()

,device_type,user_id
0,Apple,user_6809
1,Apple,user_3636
2,Apple,user_510
3,Apple,user_3468
4,Apple,user_7036


In [29]:
devices['device_type'].unique()

array(['Apple', 'brand', 'Android', 'Unknown'], dtype=object)

In [30]:
#since we don't know what "brand" is, let's change it to "unknown" as well

In [31]:
devices["device_type"] = devices["device_type"].replace("brand", "Unknown")


In [32]:
devices['device_type'].unique()

array(['Apple', 'Unknown', 'Android'], dtype=object)

In [33]:
#creates new cleaned parquet file
devices.to_parquet("/home/neobanking2025/projects/neo_bank/cleaned_data/devices.parquet", index=False)

In [34]:
#checking that all user_ids are a standard length
lengths = devices["user_id"].astype(str).str.len()

min_len = lengths.min()
max_len = lengths.max()

print(f"Min length: {min_len}")
print(f"Max length: {max_len}")


Min length: 6
Max length: 10


In [35]:
print("User IDs with the shortest length:")
print(devices[devices["user_id"].astype(str).str.len() == min_len]["user_id"].unique())

User IDs with the shortest length:
['user_1' 'user_6' 'user_4' 'user_0' 'user_5' 'user_3' 'user_7' 'user_9'
 'user_8' 'user_2']


In [36]:
print("User IDs with the longest length:")
print(devices[devices["user_id"].astype(str).str.len() == max_len]["user_id"].unique())

User IDs with the longest length:
['user_11885' 'user_15533' 'user_11964' ... 'user_12296' 'user_19056'
 'user_12199']


In [37]:
########

In [38]:
devices.head()

,device_type,user_id
0,Apple,user_6809
1,Apple,user_3636
2,Apple,user_510
3,Apple,user_3468
4,Apple,user_7036


In [39]:
validator = context.sources.pandas_default.read_parquet("/home/neobanking2025/projects/neo_bank/cleaned_data/devices.parquet")

In [40]:
validator.expect_column_values_to_not_be_null("device_type")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19431,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [41]:
validator.expect_column_values_to_not_be_null("user_id")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19431,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [42]:
validator.expect_column_distinct_values_to_be_in_set(
    column="device_type",
    value_set=["Android","Apple","Unknown"]
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "Android",
      "Apple",
      "Unknown"
    ],
    "details": {
      "value_counts": [
        {
          "value": "Android",
          "count": 9714
        },
        {
          "value": "Apple",
          "count": 9673
        },
        {
          "value": "Unknown",
          "count": 44
        }
      ]
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [43]:
validator.expect_column_values_to_match_regex(
    column="user_id",
    regex=r"^user_"
)


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19431,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [44]:
validator.expect_column_value_lengths_to_be_between(
    column="user_id",
    min_value=6,
    max_value=10
)


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19431,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Transactions

In [45]:
trans.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_1884,REFUND,AED,1.76,COMPLETED,None,NaN,None,None,INBOUND,user_8098,2018-09-25 16:03:40.978243 UTC
1,transaction_1728,REFUND,AED,639.31,COMPLETED,None,NaN,None,None,INBOUND,user_182,2018-03-31 13:45:25.262231 UTC
2,transaction_1755782,REFUND,AED,3.49,COMPLETED,None,NaN,None,None,INBOUND,user_14271,2018-11-17 16:47:25.748975 UTC
3,transaction_1973,TOPUP,AED,6079.39,COMPLETED,None,NaN,None,None,INBOUND,user_4773,2018-08-13 21:51:28.670363 UTC
4,transaction_1716,TOPUP,AED,206.65,COMPLETED,None,NaN,None,None,INBOUND,user_1363,2018-10-30 14:34:57.479895 UTC


In [46]:
trans['transactions_type'].unique()

array(['REFUND', 'TOPUP', 'EXCHANGE', 'TRANSFER', 'CARD_PAYMENT',
       'CARD_REFUND', 'ATM', 'FEE', 'CASHBACK', 'TAX'], dtype=object)

In [47]:
trans['transactions_currency'].unique()

array(['AED', 'AUD', 'BCH', 'BGN', 'BTC', 'CAD', 'CHF', 'CZK', 'DKK',
       'ETH', 'EUR', 'GBP', 'HKD', 'HRK', 'HUF', 'ILS', 'INR', 'JPY',
       'LTC', 'MAD', 'MXN', 'NOK', 'NZD', 'PLN', 'QAR', 'RON', 'RUB',
       'SAR', 'SEK', 'SGD', 'THB', 'TRY', 'USD', 'XRP', 'ZAR'],
      dtype=object)

In [48]:
trans['amount_usd'].max()

85106453190.03

In [49]:
trans['amount_usd'].min()

0.0

In [50]:
trans.dtypes

transaction_id            object
transactions_type         object
transactions_currency     object
amount_usd               float64
transactions_state        object
ea_cardholderpresence     object
ea_merchant_mcc          float64
ea_merchant_city          object
ea_merchant_country       object
direction                 object
user_id                   object
created_date              object
dtype: object

In [51]:
trans['created_date'] = pd.to_datetime(trans['created_date'],errors='coerce').dt.tz_localize(None)


In [52]:
trans = trans.dropna(subset=['created_date'])

In [53]:
trans.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_1884,REFUND,AED,1.76,COMPLETED,None,NaN,None,None,INBOUND,user_8098,2018-09-25 16:03:40.978243
1,transaction_1728,REFUND,AED,639.31,COMPLETED,None,NaN,None,None,INBOUND,user_182,2018-03-31 13:45:25.262231
2,transaction_1755782,REFUND,AED,3.49,COMPLETED,None,NaN,None,None,INBOUND,user_14271,2018-11-17 16:47:25.748975
3,transaction_1973,TOPUP,AED,6079.39,COMPLETED,None,NaN,None,None,INBOUND,user_4773,2018-08-13 21:51:28.670363
4,transaction_1716,TOPUP,AED,206.65,COMPLETED,None,NaN,None,None,INBOUND,user_1363,2018-10-30 14:34:57.479895


In [54]:
trans.dtypes

transaction_id                   object
transactions_type                object
transactions_currency            object
amount_usd                      float64
transactions_state               object
ea_cardholderpresence            object
ea_merchant_mcc                 float64
ea_merchant_city                 object
ea_merchant_country              object
direction                        object
user_id                          object
created_date             datetime64[ns]
dtype: object

In [55]:
trans['transactions_state'].unique()

array(['COMPLETED', 'CANCELLED', 'DECLINED', 'PENDING', 'REVERTED',
       'FAILED'], dtype=object)

In [56]:
trans['ea_cardholderpresence'].unique()
##fix

array([None, 'FALSE', 'TRUE', 'UNKNOWN'], dtype=object)

In [57]:
trans['ea_cardholderpresence'] = trans['ea_cardholderpresence'].fillna("UNKNOWN")

In [58]:
trans['ea_cardholderpresence'].unique()


array(['UNKNOWN', 'FALSE', 'TRUE'], dtype=object)

In [59]:
trans['ea_cardholderpresence'].isnull().sum()

0

In [60]:
trans['ea_merchant_mcc'].unique()
#drop

array([  nan, 9222., 4111., 4112., 5651., 5912., 4121., 5661., 5921.,
       5411., 5941., 7221., 5942., 5943., 7991., 3612., 5944., 5945.,
       7994., 5691., 5947., 5948., 7996., 7997., 7230., 7999., 5441.,
       3750., 5964., 5712., 5968., 5969., 5971., 8021., 7512., 5977.,
       3503., 3504., 7011., 7523., 5734., 4457., 3509., 5994., 7278.,
       5999., 3512., 3640., 6513., 3641., 4722., 4215., 5499., 6011.,
       8062., 3520., 7299., 5511., 7832., 4511., 8099., 5541., 5542.,
       3543., 5039., 5811., 5812., 5813., 5814., 5047., 5309., 5311.,
       7372., 4814., 5331., 7392., 4582., 7399., 5611., 7922., 5621.,
       7929., 7933., 3583., 5631., 8299., 9399., 7941., 5641., 3079.,
       8220., 5399., 5655., 4131., 4899., 7211., 5932., 5422., 5172.,
       5940., 4411., 7995., 3615., 7998., 5699., 5192., 5451., 8011.,
       5199., 5200., 5714., 5970., 5462., 5719., 5722., 9402., 7519.,
       5732., 5733., 5735., 8398., 5993., 5995., 7538., 8050., 3642.,
       7542., 7033.,

In [61]:
uv = trans['ea_merchant_city'].unique()
print(list(uv))
#drop

[None, 'Dubai', 'tfl.gov.uk/cp', 'Stockholm', 'M?nchen-flugh', 'Abu Dhabi', 'Paris', 'Abudhabi', 'Tallinn', 'Hamburg', '800-592-8996', 'Vorden', 'help.uber.com', 'Ras Alkhaimah', 'Dxb', 'Ras Al Khaima', 'Sharjah', 'Reykjanesbaer', 'Amsterdam', 'Swindon', 'Madrid', 'London', 'G.co Helppay#', 'www.amazon.co', 'Muenchen', 'Rak', 'Fujairah', 'Koebenhavn V', 'Paris 5', '415-800-5959', 'St. Albans', 'groupon.ae', '35314369001', 'Auh', 'Poznan', 'Abu Dhabi Ua', '123', 'Dubai       H', 'Abu Dhabi Uae', 'Arjah      A', 'Nakheel', 'Dubai Uae', 'Dubai Mins R', 'Fuj', 'Bai        A', 'Kielce', '8004049723', 'Koebenhavn K', 'Bucuresti', 'Stockholmarla', 'Ajman', '97165554545', 'Reykjavik', 'M?nchen', 'Dublin', 'Horley 012937', 'Fiumicino', 'Stockholm-arl', '03456 049 049', 'Siauliai', 'Elk Grove Vil', 'g.co/helppay#', 'Foster City', 'Newmarket', 'Sydney', 'Castle Hill', 'Melbourne', 'Australia', 'University Of', 'Hong Kong', 'Brisbane', 'Burnley', 'Chippendale', 'Subiaco', 'Mosman Park', 'South Bri

In [62]:
trans['ea_merchant_country'].unique()


array([None, 'ARE', 'GBR', 'SWE', 'DEU', 'FRA', 'EST', 'NLD', 'POL',
       'ISL', 'ESP', 'USA', 'LUX', 'DNK', 'IRL', 'JOR', 'ROM', 'ITA',
       'LTU', 'AUS', 'HKG', 'SGP', 'NZL', 'FJI', 'IDN', 'CHE', 'MYS',
       'CHN', 'VNM', 'BGR', 'MNE', 'NOR', 'ISR', 'BEL', 'CYP', 'CZE',
       'ROU', 'GIB', 'EGY', 'PRT', 'CAN', 'ZAF', 'MEX', 'COL', 'PER',
       'ARG', 'HUN', 'LKA', 'BOL', 'BRA', 'LVA', 'THA', 'RUS', 'FIN',
       'AUT', 'HRV', 'GRC', 'JPN', 'REU', 'SRB', 'BIH', 'CHL', 'TTO',
       'LBN', 'IND', 'KAZ', 'CRI', 'TWN', 'TUR', 'URY', 'AND', 'QAT',
       'LIE', 'BHS', 'MAR', 'MUS', 'OMN', 'MLT', 'BMU', 'UGA', 'ARM',
       'SYC', 'PHL', 'KEN', 'MCO', 'MDV', 'UZB', 'GEO', 'ETH', 'MOZ',
       'TCA', 'NPL', 'JAM', 'CUB', 'LAO', 'UKR', 'KHM', 'SVN', 'VCT',
       'SVK', 'BLR', 'PAN', 'MMR', 'DOM', 'TZA', 'ECU', 'MKD', 'VAT',
       'HND', 'GTM', 'PYF', 'KOR', 'SXM', 'MTQ', 'PRI', 'GLP', 'MNG',
       'TUN', 'NGA', 'AGO', 'VEN', 'SAU', 'NER', 'ATG', 'MDA', 'MYT',
       'PRY', 'SEN', 

In [63]:
trans['ea_merchant_country'] = trans['ea_merchant_country'].fillna("UNKNOWN")

In [64]:
trans['direction'].unique()


array(['INBOUND', 'OUTBOUND'], dtype=object)

In [65]:
#drop some columns we don't need
trans = trans.drop(["ea_merchant_city", "ea_merchant_mcc"], axis=1)


In [66]:
#creates new clean file
trans.to_parquet("/home/neobanking2025/projects/neo_bank/cleaned_data/transactions.parquet", index=False)

In [67]:
#######


In [68]:
validator = context.sources.pandas_default.read_parquet("/home/neobanking2025/projects/neo_bank/cleaned_data/transactions.parquet")

In [69]:
trans.head(2)

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_country,direction,user_id,created_date
0,transaction_1884,REFUND,AED,1.76,COMPLETED,UNKNOWN,UNKNOWN,INBOUND,user_8098,2018-09-25 16:03:40.978243
1,transaction_1728,REFUND,AED,639.31,COMPLETED,UNKNOWN,UNKNOWN,INBOUND,user_182,2018-03-31 13:45:25.262231


In [70]:
for col in ["created_date", "user_id", "transaction_id","transactions_type","transactions_currency","amount_usd","transactions_state","ea_cardholderpresence","ea_merchant_country"]:
    result = validator.expect_column_values_to_not_be_null(col)
    print(f"Expectation for '{col}':", result)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'created_date': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "created_date",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'user_id': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "user_id",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Expectation for 'transactions_state': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "transactions_state",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'ea_cardholderpresence': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "ea_cardholderpresence",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'ea_merchant_country': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "ea_merchant_country",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [71]:
trans['created_date'].min()

Timestamp('2018-01-01 08:51:10.248709')

In [72]:
trans['created_date'].max()

Timestamp('2019-05-16 18:22:16.959896')

In [73]:
trans['created_date'].isnull().sum()

0

## Users

In [74]:
users.head()

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals
0,user_2811,1988,IE,C,2018-03-04 21:46:39.616834 UTC,0,STANDARD,NaN,NaN,0,0,0
1,user_4750,1987,FR,Ry,2018-04-07 06:49:18.345736 UTC,0,STANDARD,NaN,NaN,0,0,0
2,user_17686,1984,GB,Ayr,2018-12-04 10:28:02.653147 UTC,0,PREMIUM,NaN,NaN,6,0,0
3,user_18779,1965,GB,Ayr,2018-12-31 07:42:19.353613 UTC,0,STANDARD,NaN,NaN,0,0,0
4,user_7823,1999,PL,Buk,2018-06-07 03:22:40.590209 UTC,0,STANDARD,NaN,NaN,0,0,0


In [75]:
users['birth_year'].min()

1929

In [76]:
users['birth_year'].max()

2001

In [77]:
users.drop(columns={'city'},inplace=True)

In [78]:
users['plan'].unique()

array(['STANDARD', 'PREMIUM', 'METAL', 'METAL_FREE', 'PREMIUM_OFFER',
       'PREMIUM_FREE'], dtype=object)

In [79]:
users['user_settings_crypto_unlocked'].unique()

array([0, 1])

In [80]:
users['attributes_notifications_marketing_push'].unique()

array([nan,  1.,  0.])

In [81]:
users['attributes_notifications_marketing_push'].value_counts()

attributes_notifications_marketing_push
1.0    12148
0.0      672
Name: count, dtype: int64

In [82]:
users['attributes_notifications_marketing_push'].isnull().sum()

6610

In [83]:
users['attributes_notifications_marketing_push'] = users['attributes_notifications_marketing_push'].round().astype('Int64')


In [84]:
users['attributes_notifications_marketing_email'].unique()

array([nan,  0.,  1.])

In [85]:
users['attributes_notifications_marketing_email'].value_counts()

attributes_notifications_marketing_email
1.0    11486
0.0     1334
Name: count, dtype: int64

In [86]:
users['attributes_notifications_marketing_email'].isnull().sum()

6610

In [87]:
users['attributes_notifications_marketing_email'] = users['attributes_notifications_marketing_email'].round().astype('Int64')


In [88]:
users['num_contacts'].min()

0

In [89]:
users['num_contacts'].max()

2918

In [90]:
users['num_referrals'].unique()

array([0])

In [91]:
users['num_referrals'].min()

0

In [92]:
users['num_referrals'].max()

0

In [93]:
users.drop(columns={'num_referrals'},inplace=True)

In [94]:
users['num_successful_referrals'].min()

0

In [95]:
users['num_successful_referrals'].max()

0

In [96]:
users.drop(columns={'num_successful_referrals'},inplace=True)

In [97]:
users['country'].unique()

array(['IE', 'FR', 'GB', 'PL', 'ES', 'NO', 'BE', 'SI', 'IT', 'RO', 'CH',
       'DE', 'CZ', 'HU', 'GR', 'NL', 'MT', 'SE', 'PT', 'LT', 'FI', 'LV',
       'EE', 'AT', 'CY', 'HR', 'LU', 'BG', 'DK', 'GG', 'GP', 'JE', 'AU',
       'SK', 'LI', 'IM', 'IS', 'GI', 'MQ', 'GF', 'RE'], dtype=object)

In [98]:
users['created_date'] = pd.to_datetime(users['created_date'],errors='coerce').dt.tz_localize(None)


In [99]:
users['created_date'].min()

Timestamp('2018-01-01 08:42:24.799709')

In [100]:
users['created_date'].max()

Timestamp('2019-01-03 07:34:36.638892')

In [101]:
#creates new clean file
users.to_parquet("/home/neobanking2025/projects/neo_bank/cleaned_data/users.parquet", index=False)

In [102]:
#####

In [103]:
users.columns

Index(['user_id', 'birth_year', 'country', 'created_date',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts'],
      dtype='object')

In [104]:
validator = context.sources.pandas_default.read_parquet("/home/neobanking2025/projects/neo_bank/cleaned_data/users.parquet")

In [105]:
for col in ["user_id", "birth_year","country","created_date","user_settings_crypto_unlocked","plan","num_contacts"]:
    result = validator.expect_column_values_to_not_be_null(col)
    print(f"Expectation for '{col}':", result)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'user_id': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "user_id",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'birth_year': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "birth_year",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Expectation for 'user_settings_crypto_unlocked': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "user_settings_crypto_unlocked",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'plan': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "plan",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'num_contacts': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "num_contacts",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [106]:
validator.expect_column_values_to_be_unique(column='user_id')


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Sending Cleaned Data to GCP Cloud Storage Bucket

In [ ]:
# from google.cloud import storage

In [108]:
def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to the GCS bucket."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_path)
    print(f"Uploaded {source_file_path} to gs://{bucket_name}/{destination_blob_name}")

In [ ]:
# upload_to_gcs(
#     bucket_name="neobank_data_bucket",
#     source_file_path="/home/neobanking2025/projects/neo_bank/cleaned_data/transactions.parquet",
#     destination_blob_name="staging_data/"
# )

Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/neobank_data_bucket/o?uploadType=resumable: {
  "error": {
    "code": 403,
    "message": "Provided scope(s) are not authorized",
    "errors": [
      {
        "message": "Provided scope(s) are not authorized",
        "domain": "global",
        "reason": "forbidden"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.CREATED: 201>)